In [1]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))

cuda_malloc_async


In [2]:
import numpy as np 
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore")

In [3]:
from tensorflow.keras import datasets, layers, models, callbacks
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.regularizers import l2
from keras.layers import Conv2D
from keras.regularizers import l1
from keras.regularizers import l1_l2
from keras import regularizers

In [4]:
train_datagen_with_transforms = tf.keras.preprocessing.image.ImageDataGenerator(      #tranforming the data for data augmentation
    rescale=1. / 255,rotation_range=10, fill_mode='nearest',
     featurewise_center=True,
    featurewise_std_normalization=True,
    vertical_flip= True,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True )
train_datagen_with_no_transforms = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0/255)

train_generator = train_datagen_with_no_transforms.flow_from_directory(          
    "/mnt/c/Users/aniru/Desktop/train",
    target_size=(200, 200),
    batch_size=32,
    class_mode="categorical"
)
train_generator_with_data_augmentation = train_datagen_with_transforms.flow_from_directory(
    "/mnt/c/Users/aniru/Desktop/train",
    target_size=(200, 200),
    batch_size=32,
    class_mode="categorical"
)
validation_generator = val_datagen.flow_from_directory(
    "/mnt/c/Users/aniru/Desktop/test",
    target_size=(200,200),
    batch_size=32,
    class_mode="categorical"
    
)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2) 

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


train_data = train_datagen.flow_from_directory("/content/drive/MyDrive/intern/train", 
                                               target_size=(200, 200), 
                                               color_mode='rgb', 
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset = 'training')

val_data = val_datagen.flow_from_directory("/content/drive/MyDrive/intern/train", 
                                           target_size=(200, 200), 
                                           color_mode='rgb', 
                                           class_mode='categorical',
                                           shuffle=False,
                                           subset = 'validation')

Found 1877 images belonging to 28 classes.
Found 455 images belonging to 28 classes.


In [7]:
sample_data = next(val_data)

In [8]:
sample_data[0].shape   #shape of our images

(32, 200, 200, 3)

In [9]:
nb_train_samples = train_data.samples
nb_val_samples = val_data.samples

In [10]:
train_data.samples

1877

# CNN

In [11]:
model = models.Sequential() 
model.add(tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(200,200,3), padding="same"))
model.add(keras.layers.MaxPool2D(2,2))
model.add(tf.keras.layers.Conv2D(32, (3,3), activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2,2))
model.add(tf.keras.layers.Flatten())                                   
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(28,activation="Softmax"))

In [22]:
#! pip install hyperas

In [23]:
#pip install -U keras-tuner

In [24]:
from kerastuner.tuners import RandomSearch

In [32]:
def buildmodel(hp):
  model = models.Sequential() 
  model.add(tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(200,200,3), padding="same"))
  model.add(keras.layers.MaxPool2D(2,2))
  model.add(tf.keras.layers.Conv2D(32, (3,3), activation="relu", padding="same"))
  model.add(keras.layers.MaxPool2D(2,2))
  model.add(tf.keras.layers.Flatten()) 
  model.add(layers.Dense(units=hp.Int('units', min_value=32,max_value=512,step=128),activation='relu'))
  model.add(tf.keras.layers.Dense(28,activation="Softmax"))
  loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  optimizer_fn = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer_fn,
              loss=loss_fn,
              metrics=['accuracy'])
  return model

In [34]:
tuner=RandomSearch(
    buildmodel,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3
)

In [35]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 128, 'sampling': None}


In [ ]:
tuner.search(train_data,epochs=5,validation_data=val_data)

Trial 1 Complete [00h 48m 42s]
val_accuracy: 0.05494505539536476

Best val_accuracy So Far: 0.05494505539536476
Total elapsed time: 00h 48m 42s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
288               |416               |units

Epoch 1/5
59/59 [==============================] - 175s 3s/step - loss: 3.3464 - accuracy: 0.0357 - val_loss: 3.3332 - val_accuracy: 0.0396
Epoch 2/5
59/59 [==============================] - 177s 3s/step - loss: 3.3168 - accuracy: 0.0719 - val_loss: 3.3132 - val_accuracy: 0.0769
Epoch 3/5
57/59 [===========================>..] - ETA: 5s - loss: 3.3137 - accuracy: 0.0756

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 100, 100, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 50, 50, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 80000)             0         
                                                                 
 dense (Dense)               (None, 64)                5

In [28]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [29]:
optimizer_fn = tf.keras.optimizers.Adam(learning_rate=0.001)

In [30]:
model.compile(optimizer=optimizer_fn,
              loss=loss_fn,
              metrics=['accuracy'])

In [31]:
history=model.fit_generator(train_data, epochs = 10, validation_data = val_data)

Epoch 1/10
59/59 [==============================] - 209s 3s/step - loss: 3.3150 - accuracy: 0.0767 - val_loss: 3.3148 - val_accuracy: 0.0769
Epoch 2/10
59/59 [==============================] - 179s 3s/step - loss: 3.3150 - accuracy: 0.0767 - val_loss: 3.3148 - val_accuracy: 0.0769
Epoch 3/10
59/59 [==============================] - 171s 3s/step - loss: 3.3150 - accuracy: 0.0767 - val_loss: 3.3148 - val_accuracy: 0.0769
Epoch 4/10
59/59 [==============================] - 172s 3s/step - loss: 3.3150 - accuracy: 0.0767 - val_loss: 3.3148 - val_accuracy: 0.0769
Epoch 5/10
59/59 [==============================] - 173s 3s/step - loss: 3.3150 - accuracy: 0.0767 - val_loss: 3.3148 - val_accuracy: 0.0769
Epoch 6/10
59/59 [==============================] - 186s 3s/step - loss: 3.3150 - accuracy: 0.0767 - val_loss: 3.3148 - val_accuracy: 0.0769
Epoch 7/10
59/59 [==============================] - 170s 3s/step - loss: 3.3150 - accuracy: 0.0767 - val_loss: 3.3148 - val_accuracy: 0.0769
Epoch 8/10
59

KeyboardInterrupt: ignored

# with augmentation 

In [ ]:
model2 =models.Sequential()     
model2.add(tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(200,200,3), padding="same"))
model2.add(tf.keras.layers.MaxPooling2D(2,2))
model2.add(tf.keras.layers.Conv2D(32, (3,3), activation="relu"))
model2.add(tf.keras.layers.Conv2D(64, (3,3), activation="relu"))
model2.add(keras.layers.MaxPooling2D(2,2))
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dropout(0.4))
model2.add(tf.keras.layers.Dense(64, activation="relu"))
model2.add(tf.keras.layers.Dense(28,activation="Softmax"))